In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

In [4]:
## Read the pdfs from the folder
loader = PyPDFDirectoryLoader("./us_census")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
final_docs = text_splitter.split_documents(docs)
final_docs[0]

Document(page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015Issued September 2023Douglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to health coverage. For example, between 2021 and 2022, the labor market continued to improve, which may have affected private coverage in the United States \nduring that time.\n1 Public policy changes included \nthe renewal of the Public Health Emergency, which \nallowed Medicaid enrollees to remain covered under the Continuous Enrollment Provision.\n2 The American \nRescue Plan (ARP) enhanced Marketplace premium subsidies for those with incomes above 400 percent of the poverty level as well as for unemployed people.\n3', metadata={'source': 'us_census\\acsbr-015.pdf', 'page': 0})

In [5]:
len(final_docs)

316

In [14]:
## Embedding Using Huggingface
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",
    model_kwargs={"device":"cpu"},
    encode_kwargs={"normalize_embeddings":True}
)

c:\Users\chun\anaconda3\envs\LC\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [15]:
import numpy as np
np.array(huggingface_embeddings.embed_query(final_docs[0].page_content))


array([-8.46568793e-02, -1.19099170e-02, -3.37892920e-02,  2.94559058e-02,
        5.19159809e-02,  5.73840030e-02, -4.10017446e-02,  2.74267811e-02,
       -1.05128214e-01, -1.58056132e-02,  7.94858560e-02,  5.64318411e-02,
       -1.31765492e-02, -3.41544114e-02,  5.81604149e-03,  4.72548120e-02,
       -1.30746895e-02,  3.12990299e-03, -3.44225466e-02,  3.08406278e-02,
       -4.09085974e-02,  3.52738164e-02, -2.43761465e-02, -4.35831547e-02,
        2.41503213e-02,  1.31986653e-02, -4.84451978e-03,  1.92347635e-02,
       -5.43912798e-02, -1.42735019e-01,  5.15527837e-03,  2.93115564e-02,
       -5.60810938e-02, -8.53537861e-03,  3.14141549e-02,  2.76736338e-02,
       -2.06188150e-02,  8.24231356e-02,  4.15425263e-02,  5.79655208e-02,
       -3.71586867e-02,  6.26157969e-03, -2.41390150e-02, -5.61794871e-03,
       -2.51715239e-02,  5.04968641e-03, -2.52801068e-02, -2.91944365e-03,
       -8.24044459e-03, -5.69604151e-02,  2.30823029e-02, -5.54218842e-03,
        5.11555709e-02,  

In [16]:
## Vectorstore creation
vectorstore = FAISS.from_documents(final_docs, huggingface_embeddings)

In [17]:
## Query using Similarity Search
query = "What is the health insurance coverage?"
relevant_docments = vectorstore.similarity_search(query)

print(relevant_docments[0].page_content)

2 U.S. Census Bureau
WHAT IS HEALTH INSURANCE COVERAGE?
This brief presents state-level estimates of health insurance coverage 
using data from the American Community Survey (ACS). The  
U.S. Census Bureau conducts the ACS throughout the year; the 
survey asks respondents to report their coverage at the time of 
interview. The resulting measure of health insurance coverage, 
therefore, reflects an annual average of current comprehensive 
health insurance coverage status.* This uninsured rate measures a 
different concept than the measure based on the Current Population 
Survey Annual Social and Economic Supplement (CPS ASEC). 
For reporting purposes, the ACS broadly classifies health insurance 
coverage as private insurance or public insurance. The ACS defines 
private health insurance as a plan provided through an employer 
or a union, coverage purchased directly by an individual from an 
insurance company or through an exchange (such as healthcare.


In [18]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000020435E512A0> search_kwargs={'k': 3}


In [34]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [ ]:
from langchain_community.llms import HuggingFaceHub

hf = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={"temperature":0.1, "max_length":500}
)

query = "What is the health insurance coverage?"
hf.invoke(query)

In [ ]:
#Hugging Face models can be run locally through the HuggingFacePipeline class.
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id="mistralai/Mistral-7B-v0.1",
    task="text-generation",
    pipeline_kwargs={"temperature": 0, "max_new_tokens": 300}
)

llm = hf 
llm.invoke(query)

In [41]:
prompt_template = """
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context:
<context>
{context}
</context>
Question:{question}
Helpful Answers:
 """

In [42]:
prompt = PromptTemplate(template=prompt_template, input_variables=["context","question"])

In [43]:
retrievalQA = RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt":prompt}
)

In [ ]:
# Call the QA chain with our query.
query = """DIFFERENCES IN THE UNINSURED RATE BY STATE IN 2022"""
result = retrievalQA.invoke({"query": query})
print(result['result'])